In [ ]:
import numpy as np
import scipy
import scipy.io
import scipy.sparse as sp
import csv

#import surprise

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

%load_ext autoreload
%autoreload 2
%matplotlib inline

from helpers import calculate_mse

# 0.1 Load and prepare data

### Training and testing data

#### Load the Data
`ratings` is a sparse matrix in the shape of (num_items, num_users)

In [ ]:
from helpers import load_data, preprocess_data

DATA_PATH = '../data/'
PREDICTION_PATH = '../data/predictions/'
ratings = load_data('{dp}data_train.csv'.format(dp=DATA_PATH))
#ratings = load_data('{dp}movielens100k.csv'.format(dp=DATA_PATH))

#### Plot the number of ratings per movie and user

In [ ]:
from plots import plot_raw_data

num_items_per_user, num_users_per_item = plot_raw_data(ratings)

print("min # of items per user = {}, min # of users per item = {}.".format(
        min(num_items_per_user), min(num_users_per_item)))

#### Split the data into a train and test set

In [ ]:
def split_data(ratings, num_items_per_user, num_users_per_item,
               min_num_ratings, p_test=0.1, verbose=False):
    """split the ratings to training data and test data.
    Args:
        min_num_ratings: 
            all users and items we keep must have at least min_num_ratings per user and per item. 
    """
    # set seed
    np.random.seed(988)
    
    # select user and item based on the condition.
    valid_users = np.where(num_items_per_user >= min_num_ratings)[0]
    valid_items = np.where(num_users_per_item >= min_num_ratings)[0]
    valid_ratings = ratings[valid_items, :][:, valid_users]
    
    # LIL is a convenient format for constructing sparse matrices
    train = sp.lil_matrix(valid_ratings.shape)
    test = sp.lil_matrix(valid_ratings.shape)
    
    valid_ratings_i, valid_ratings_u, valid_ratings_v = sp.find(valid_ratings)
    valid_ratings_p_idx = np.random.permutation(range(len(valid_ratings_i)))
    
    n_test = int(p_test*len(valid_ratings_i))
    
    for idx in valid_ratings_p_idx[:n_test]:
        test[valid_ratings_i[idx], valid_ratings_u[idx]] = valid_ratings_v[idx]
        
    for idx in valid_ratings_p_idx[n_test:]:
        train[valid_ratings_i[idx], valid_ratings_u[idx]] = valid_ratings_v[idx]

    if verbose:
        print("Total number of nonzero elements in original data:{v}".format(v=ratings.nnz))
        print("Total number of nonzero elements in train data:{v}".format(v=train.nnz))
        print("Total number of nonzero elements in test data:{v}".format(v=test.nnz))
    
    # convert to CSR for faster operations
    return valid_ratings, train.tocsr(), test.tocsr()

In [ ]:
valid_ratings, train, test = split_data(ratings, num_items_per_user,
    num_users_per_item, min_num_ratings=10, p_test=0.1, verbose=True)

### Submission creation data

In [ ]:
ratings_csr = ratings.tocsr()
sample_submission = load_data('{dp}sample_submission.csv'.format(dp=DATA_PATH))
sample_submission_csr = sample_submission.tocsr()

# 0.2 Auxiliary functions

In [ ]:
def compute_division(a, b):
    """Computes element by element division.
    If x/0 returns 0.
    """
    # Raises error if vectors have different lengths
    assert(len(a) == len(b))
    
    # Computes division
    res = a.copy()
    for i in range(len(a)):
        if b[i] == 0:
            res[i] = 0
        else:
            res[i] = a[i] / b[i]

    return res

# 1 Baselines

## 1.1 Baselines implemented from scratch

In [ ]:
# Baseline rating
def baseline_rating(data):
    """Implements baseline method for a ratings matrix
    using the global mean.
    """
    # Compute global mean using training data
    r_mean = data.sum() / data.count_nonzero()
    return r_mean


# User or item specific effect
def baseline_user_item_specific(data, mean, set_num=0):
    """Implements baseline method for a ratings matrix
    using either the user or the item mean,
    as indicated in parameter mean.
    """
    if mean=="user":
        flag = 1
        inv_flag = 0
    else:
        flag = 0
        inv_flag = 1

    num = max(set_num, data.shape[flag])
    
    # Obtain r_demeaned (ratings minus global avg)
    global_mean = baseline_rating(data)
    r_demeaned = data.copy()
    r_demeaned.data = (1.0 * r_demeaned.data) - global_mean
    
    # Compute means using training data
    # get rows, columns and values for elements in r_demeaned
    data_rcv = sp.find(r_demeaned)
    # compute means
    counts = np.bincount(data_rcv[flag], minlength=num)
    sums = np.bincount(data_rcv[flag], weights=data_rcv[2], minlength=num)
    means = compute_division(sums, counts)

    return means

We first estimate the RMSE for our test set:

In [ ]:
def model_baseline(data, test_data, test_flag, sub_flag=False,
    sub_filename="new_submission", verbose=False):

    """If 'test_flag' is True, then 'data' should be the training dataset
    'test_data' the test dataset. In this case sub_flag is ignored.
    
    If 'test_flag' is False and 'sub_flag' is True, then 'data' should be
    the entire ratings dataset and 'test_data' should be a sample submission.
    
    Both 'data' and 'test_data' should be csr sparse matrices.
    """
    assert test_flag or sub_flag, "Specify a task"
    
    num_train_items, num_train_users = data.shape
    num_test_items, num_test_users = test_data.shape
    
    num_i_max = max(num_train_items, num_test_items)
    num_u_max = max(num_train_users, num_test_users)
    
    global_mean = baseline_rating(data)
    item_means = baseline_user_item_specific(data, 'item', set_num=num_i_max)
    user_means = baseline_user_item_specific(data, 'user', set_num=num_u_max)
    
    (rows, cols, vals) = sp.find(test_data)
    
    if test_flag:        
        # Do predictions
        pred_test = np.array([(global_mean + item_means[i] + user_means[u])
            for (i, u) in zip(rows, cols)])

        # Compute and print test error
        test_mse = calculate_mse(vals, pred_test)
        test_rmse = np.sqrt(test_mse / len(vals))
        if verbose:
            print("Test RMSE of baseline using baseline: {e}".format(e=test_rmse)) 
        return test_rmse, pred_test

    elif sub_flag:
        # Directly write predictions to submission file
        with open('{dp}{fn}.csv'.format(dp=PREDICTION_PATH, fn=sub_filename), 'w') as csvfile:
            fieldnames = ['Id', 'Prediction']
            writer = csv.DictWriter(csvfile, delimiter=",", fieldnames=fieldnames)
            writer.writeheader()
            for (i, u) in zip(rows, cols):
                pred_i_u = global_mean + user_means[u] + item_means[i]
                writer.writerow({'Id':'r{r}_c{c}'.format(r=i+1,c=u+1),'Prediction':pred_i_u})

Now we prepare the submission file training on all data:

# 2 Matrix Factorization

## 2.1 Matrix Factorization with scipy's svds (uses 1.1)

In [ ]:
def demean_matrix(data, verbose=False):
    """Removes the global, user and item means from a matrix.
    Returns the matrix and the computed means.
    """
    num_rows, num_cols = data.shape
    (rows, cols, vals) = sp.find(data)
    
    # Compute global, user and item means    
    global_mean = baseline_rating(data)
    item_means = baseline_user_item_specific(data, 'item')
    user_means = baseline_user_item_specific(data, 'user')
    
    # Substract the baseline of each element in 'data'
    train_vals = vals.copy()
    train_vals = 1.0 * train_vals
    
    baselines = np.array([(global_mean + item_means[i] + user_means[u])
        for (i, u) in zip(rows, cols)])
    train_vals = train_vals - baselines
    
    #for (i, u) in zip(rows, cols):
    #    baseline_i_u = global_mean + user_means[u] + item_means[i]
    #    train_vals[(rows==i) & (cols==u)] += (- baseline_i_u)

    # Get matrix
    r_demeaned = sp.csr_matrix((train_vals, (rows, cols)),
        shape=(num_rows, num_cols))
    
    if verbose:
        print('---------------------------------------------')
        print('          Completed demean_matrix!           ')
        print('---------------------------------------------')
    
    return r_demeaned, global_mean, user_means, item_means

In [ ]:
from scipy.sparse.linalg import svds

def model_matrix_factorization(data, test_data, test_flag, k=20, sub_flag=False,
    sub_filename="new_submission", alpha=0, l1_ratio=0, shuffle=True, random_state=42,
    verbose=False):
    """If 'test_flag' is True, then 'data' should be the training dataset and
    'test_data' the test dataset. In this case sub_flag is ignored.
    
    If 'test_flag' is False and 'sub_flag' is True, then 'data' should be
    the entire ratings dataset and 'test_data' should be a sample submission.
    
    Both 'data' and 'test_data' should be csr sparse matrices.
    """

    assert test_flag or sub_flag, "Specify a task"
    assert k <= min(data.shape), "k must be smaller than the min dimension of 'data'"
    
    # Substract baseline from data
    r_demeaned, global_mean, user_means, item_means = demean_matrix(data, verbose=verbose)
    
    # Use scipy's svds
    U, sigma, Vt = svds(r_demeaned, k)
    sigma = np.diag(sigma)
    U_sigma = np.dot(U, sigma)
    # pred_matrix = np.dot(np.dot(U, sigma), Vt) # esta matriz no tiene sentido calcularla?
    
    if verbose:
        print('Finished fitting model')

    (test_rows, test_cols, test_vals) = sp.find(test_data)
    (rows, cols, vals) = sp.find(data)

    if test_flag:        
        # Do predictions        
        baselines = np.array([(global_mean + item_means[i] + user_means[u])
            for (i, u) in zip(test_rows, test_cols)])
        
        interactions = np.array([(U_sigma[i,:].dot(Vt[:,u]))
            for (i, u) in zip(test_rows, test_cols)])
        
        pred_test = baselines + interactions
    
        #for (i, u) in zip(test_rows, test_cols):
        #    mat_fact_i_u = pred_matrix[i,u]
        #    baseline_i_u = global_mean + user_means[u] + item_means[i]
        #    pred_i_u = mat_fact_i_u + baseline_i_u
        #    pred_test[(test_rows==i) & (test_cols==u)] = pred_i_u
        
        if verbose:
            print('Finished predicting')

        # Compute and print test error
        test_mse = calculate_mse(test_vals, pred_test)
        test_rmse = np.sqrt(test_mse / len(test_vals))
        
        if verbose:
            print("Test RMSE using baseline and matrix factorization: {e}".format(e=test_rmse)) 
            print()
            print('-----------------------------------------------')
            print(' Completed test in model_matrix_factorization! ')    
            print('-----------------------------------------------')
            
        return test_rmse, pred_test

    elif sub_flag:
        # Directly write predictions to submission file
        with open('{dp}{fn}.csv'.format(dp=PREDICTION_PATH, fn=sub_filename), 'w') as csvfile:
            fieldnames = ['Id', 'Prediction']
            writer = csv.DictWriter(csvfile, delimiter=",", fieldnames=fieldnames)
            writer.writeheader()
            for (i, u) in zip(test_rows, test_cols):
                interaction = U_sigma[i,:].dot(Vt[:,u])
                baseline = global_mean + user_means[u] + item_means[i]
                pred_i_u = interaction + baseline
                writer.writerow({'Id':'r{r}_c{c}'.format(r=i+1,c=u+1),'Prediction':pred_i_u})

        if verbose:
            print('-----------------------------------------------------')
            print(' Completed submission in model_matrix_factorization! ')    
            print('-----------------------------------------------------')

Test:

Now we prepare the submission file for Kaggle:

# Extra

## E.1 Very simple example to try new functions on:

In [ ]:
nrows, ncols = (4,3)
a=np.random.randint(0,7,(nrows,ncols))
a[a>5]=0
a=sp.csr_matrix(a)
print(a.todense())
print()

#### 1. Try with sklearn' s NMF (wrong approach)

In [ ]:
a_demeaned, gm, um, im = demean_matrix(a, 5)
print(a_demeaned)
print()

nmf_model = NMF(alpha=0, l1_ratio=0, shuffle=True, random_state=1)
W = nmf_model.fit_transform(a_demeaned)
H = nmf_model.components_

pred_matrix=W.dot(H)
print()
print(W)
print(H)

pred_sol = np.full((nrows,ncols),1.0)

for i in range(nrows):
    for u in range(ncols):
        mat_fact_i_u = pred_matrix[i,u]
        baseline_i_u = gm + um[u] + im[i]
        pred_i_u = mat_fact_i_u - 5 #+ baseline_i_u

        pred_sol[i,u] = pred_i_u
        #print('i{} u{} mf{} bl{} p{}'.format(i,u,mat_fact_i_u,baseline_i_u,pred_i_u))

print()
print(pred_sol)

#### 2. With scipy's SVD:

In [ ]:
print(a.todense())
print()

a_demeaned, gm, um, im = demean_matrix(a, 0)
print(a_demeaned)
print()

U, sigma, Vt = svds(a_demeaned, k = 2)
sigma = np.diag(sigma)

pred_matrix = np.dot(np.dot(U, sigma), Vt)
pred_sol = np.full((nrows,ncols),1.0)

for i in range(nrows):
    for u in range(ncols):
        mat_fact_i_u = pred_matrix[i,u]
        baseline_i_u = gm + um[u] + im[i]
        pred_i_u = mat_fact_i_u + baseline_i_u

        pred_sol[i,u] = pred_i_u
        #print('i{} u{} mf{} bl{} p{}'.format(i,u,mat_fact_i_u,baseline_i_u,pred_i_u))
print()
print(pred_sol)

## E.2 Matrix Factorization for Ex10

### Matrix Factorization using SGD

Some guy wanted to optimize running time, this is his code:

https://codereview.stackexchange.com/questions/35727/optimize-scipy-sparse-matrix-factorization-code-for-sgd

In [ ]:
N = self.model.shape[0] #no of users
M = self.model.shape[1] #no of items
self.p = np.random.rand(N, K)
self.q = np.random.rand(M, K)
rows,cols = self.model.nonzero()        
for step in xrange(steps):
    for u, i in zip(rows,cols):
        e = self.model[u, i] - np.dot(self.p[u, :], self.q[i, :]) #calculate error for gradient
        p_temp = learning_rate * ( e * self.q[i,:] - regularization * self.p[u,:])
        self.q[i,:]+= learning_rate * ( e * self.p[u,:] - regularization * self.q[i,:])
        self.p[u,:] += p_temp

#### Initialize matrix factorization

In [ ]:
def init_MF(train, num_features):
    """init the parameter for matrix factorization."""
    
    # ***************************************************
    # INSERT YOUR CODE HERE
    # TODO
    # you should return:
    #     user_features: shape = num_features, num_user
    #     item_features: shape = num_features, num_item
    # ***************************************************
    return user_features, item_features

Compute the cost by the method of matrix factorization.


In [ ]:
def compute_error(data, user_features, item_features, nz):
    """compute the loss (MSE) of the prediction of nonzero elements."""
    # ***************************************************
    # INSERT YOUR CODE HERE
    # TODO
    # calculate rmse (we only consider nonzero entries.)
    # ***************************************************
    raise NotImplementedError

In [ ]:
def matrix_factorization_SGD(train, test):
    """matrix factorization by SGD."""
    # define parameters
    gamma = 0.01
    num_features = 20   # K in the lecture notes
    lambda_user = 0.1
    lambda_item = 0.7
    num_epochs = 20     # number of full passes through the train set
    errors = [0]
    
    # set seed
    np.random.seed(988)

    # init matrix
    user_features, item_features = init_MF(train, num_features)
    
    # find the non-zero ratings indices 
    nz_row, nz_col = train.nonzero()
    nz_train = list(zip(nz_row, nz_col))
    nz_row, nz_col = test.nonzero()
    nz_test = list(zip(nz_row, nz_col))

    print("learn the matrix factorization using SGD...")
    for it in range(num_epochs):        
        # shuffle the training rating indices
        np.random.shuffle(nz_train)
        
        # decrease step size
        gamma /= 1.2
        
        for d, n in nz_train:
        # ***************************************************
        # INSERT YOUR CODE HERE
        # TODO
        # do matrix factorization.
        # ***************************************************
        raise NotImplementedError

        print("iter: {}, RMSE on training set: {}.".format(it, rmse))
        
        errors.append(rmse)
    # ***************************************************
    # TODO
    # evaluate the test error.
    # ***************************************************
    rmse = compute_error(test, user_features, item_features, nz_test)
    print("RMSE on test data: {}.".format(rmse))
    raise NotImplementedError

matrix_factorization_SGD(train, test)   

### Learn the Matrix Factorization using Alternating Least Squares

In [ ]:
def update_user_feature(
        train, item_features, lambda_user,
        nnz_items_per_user, nz_user_itemindices):
    """update user feature matrix."""
    # ***************************************************
    # INSERT YOUR CODE HERE
    # TODO
    # update and return user feature.
    # ***************************************************
    raise NotImplementedError

def update_item_feature(
        train, user_features, lambda_item,
        nnz_users_per_item, nz_item_userindices):
    """update item feature matrix."""
    # ***************************************************
    # INSERT YOUR CODE HERE
    # TODO
    # update and return item feature.
    # ***************************************************
    raise NotImplementedError

In [ ]:
from helpers import build_index_groups


def ALS(train, test):
    """Alternating Least Squares (ALS) algorithm."""
    # define parameters
    num_features = 20   # K in the lecture notes
    lambda_user = 0.1
    lambda_item = 0.7
    stop_criterion = 1e-4
    change = 1
    error_list = [0, 0]
    
    # set seed
    np.random.seed(988)

    # init ALS
    user_features, item_features = init_MF(train, num_features)
    
    # ***************************************************
    # INSERT YOUR CODE HERE
    # TODO
    # start you ALS-WR algorithm.
    # ***************************************************
    raise NotImplementedError

ALS(train, test)